# Import

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
from app.utils.functions import *
from app.core import config
from IPython.display import HTML
from app.services.ollama_service import format_response
import httpx

# Data Preparation

In [25]:
def preprocessing_data(df: pd.DataFrame, simple_dict: list[dict], colonne_type: str = "Année contexte") -> pd.DataFrame:
    """Prétraite et catégorise le DataFrame selon les groupes définis."""
    try:
        required_columns = ['Lignes', 'Contexte', 'Nature de l\'écriture', 'Année', 'Mois', 'Montant']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"La colonne requise '{col}' est absente du DataFrame.")
        mask = (
            (df.iloc[:, 0] == df.iloc[0, 0]) &
            (df.iloc[:, 2] == "Compte d'exploitation")
        )
        df_filtered: pd.DataFrame
        df_filtered = df[mask].copy()
        
        if df_filtered.empty:
            df_filtered['Groupe'] = pd.Series(dtype='object')
            return df_filtered

        mask_pct_recettes = df_filtered['Lignes'] == "% DES RECETTES TOTALES"
        df_pct_recettes = df_filtered[mask_pct_recettes].copy()
        df_autres = df_filtered[~mask_pct_recettes].copy()

        if not df_pct_recettes.empty:
            sort_keys = ['Année', 'Mois', 'Contexte', "Nature de l'écriture", 'Montant']
            existing_sort_keys = [col for col in sort_keys if col in df_pct_recettes.columns]
            df_pct_recettes = (
                df_pct_recettes
                .sort_values(by=existing_sort_keys[:-1] + ['Montant'], ascending=[True]*len(existing_sort_keys[:-1])+[False])
                .reset_index(drop=True)
            )
            code_map = {0: "4.", 1: "7.", 2: "10."}
            df_pct_recettes['Code Hiérarchique'] = (
                df_pct_recettes
                .groupby(existing_sort_keys[:-1], sort=False)
                .cumcount()
                .map(lambda idx: code_map.get(idx, None))
            )

        coded_tree = generate_hierarchy_codes(simple_dict)
        hierarchy_list = extract_flat_hierarchy_list(coded_tree)
        mapping_dict = {item['label']: item['code'] for item in hierarchy_list}

        if not df_autres.empty:
            df_autres['Code Hiérarchique'] = df_autres['Lignes'].map(mapping_dict)
        df_filtered = pd.concat([df_autres, df_pct_recettes]).sort_index(kind="stable")
        cols = list(df.columns)
        if "Code Hiérarchique" in cols and "Lignes" in cols:
            cols.remove("Code Hiérarchique")
            insert_idx = cols.index("Lignes")
            cols = cols[:insert_idx] + ["Code Hiérarchique"] + cols[insert_idx:]
            df_filtered = df_filtered[cols]

        result_list = extract_all_descendants_for_list(simple_dict)
        CHIFFRE_AFFAIRES = result_list[0] + result_list[3] + result_list[6] + result_list[9] + result_list[10]
        CHARGES = result_list[1] + result_list[4] + result_list[7] + result_list[12]
        MARGES = result_list[2] + result_list[5] + result_list[8] + result_list[11] + result_list[13]
        groupes_dict = {
            "Chiffre d'affaire": CHIFFRE_AFFAIRES,
            "Charge": CHARGES,
            "Marge": MARGES
        }
        groupes_mapping = {poste: groupe for groupe, postes in groupes_dict.items() for poste in postes}
        df_filtered['Groupe'] = df_filtered['Lignes'].map(groupes_mapping)

        mask_pct = (df_filtered['Lignes'] == "% DES RECETTES TOTALES") | (df_filtered['Colonnes'].str.contains('%', na=False))
        mask_non_pct = ~mask_pct

        # Correction : bien gérer le typage et l'arrondi sans convertir en int les NaN et préserver les floats si NaN
        df_filtered.loc[mask_non_pct, "Montant"] = df_filtered.loc[mask_non_pct, "Montant"].round(0)
        mask_non_pct_non_nan = mask_non_pct & df_filtered['Montant'].notna()
        df_filtered.loc[mask_non_pct_non_nan, "Montant"] = df_filtered.loc[mask_non_pct_non_nan, "Montant"].astype(int)

        if mask_pct.any():
            df_filtered.loc[mask_pct, "Montant"] = df_filtered.loc[mask_pct, "Montant"].round(2)
            
        df_filtered = df_filtered[df_filtered["Type de colonnes"] == colonne_type].copy()
        df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)
        return df_filtered
    except Exception as e:
        logger.error(f"Erreur lors du prétraitement des données : {e}")
        raise

In [26]:
# Chargement du fichier (en supposant la même structure que précédemment)
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)
lexiques = await get_mapping()


df = pd.read_csv(r"data.csv")
df = preprocessing_data(df, simple_dict)

In [27]:
# Renommage et nettoyage
df = df.rename(
    columns={
        'Code Hiérarchique': 'Code_H', 
        'Montant': 'Montant',
        'Lignes': 'Ligne_Analytique',
        'Contexte': 'Contexte',
        'Année': 'Annee',
        'Groupe': 'Groupe',
        'Section  analytique': 'Residence'
    }
)

df['Annee'] = df['Annee'].astype(int)
df['Mois'] = df['Mois'].astype(int)
df['Contexte'] = df['Contexte'].replace({'R': 'Réel', 'B': 'Budget', 'P': 'Prévision'})

df_agg = df.groupby(
    [
        'Residence', 'Colonnes', 'Annee', 'Mois', "Nature de l'écriture", 'Contexte', 'Code_H', 'Ligne_Analytique', 'Groupe'
    ]
)['Montant'].sum().reset_index()

contexte_order = ['Réel', 'Prévision', 'Budget']

def mois_sort_key(mois):
    try:
        return int(mois)
    except:
        return 99

df_pivot = df_agg.pivot_table(
    index=['Groupe', 'Code_H', 'Ligne_Analytique'],
    columns=['Annee', 'Contexte', 'Mois', "Nature de l'écriture"],
    values='Montant',
    fill_value=0,
    aggfunc='sum'
)

if df_pivot.columns.nlevels == 4:
    nature_unique = df_pivot.columns.get_level_values(3).unique().tolist()
    if "Annuelle" in nature_unique:
        nature_unique = [n for n in nature_unique if n != "Annuelle"]
        nature_order_desc = sorted(nature_unique, reverse=True) + ["Annuelle"]
    else:
        nature_order_desc = sorted(nature_unique, reverse=True)
    nature_order_dict = {name: i for i, name in enumerate(nature_order_desc)}
    
    def col_sort_key(x):
        return (
            int(x[0]) if str(x[0]).isdigit() else 0,
            contexte_order.index(x[1]) if x[1] in contexte_order else 99,
            mois_sort_key(x[2]),
            nature_order_dict.get(x[3], 999)
        )
    df_pivot = df_pivot[sorted(df_pivot.columns, key=col_sort_key)]

df_pivot = df_pivot.reset_index()

def code_hierarchical_sort_key(code):
    parts = [int(part) if part.isdigit() else part for part in re.split(r'\D+', str(code).strip('.')) if part]
    return parts

df_pivot_sorted = df_pivot.copy()
df_pivot_sorted['__sort_key'] = df_pivot_sorted['Code_H'].apply(code_hierarchical_sort_key)
df_pivot_sorted = df_pivot_sorted.sort_values('__sort_key').drop(columns='__sort_key').reset_index(drop=True)
mask_annuelle = df_pivot_sorted.columns.get_level_values(3) == "Annuelle"
annuelle_cols = df_pivot_sorted.columns[mask_annuelle].tolist()

meta_cols = [c for c in df_pivot_sorted.columns if c[0] in ("Groupe", "Code_H", "Ligne_Analytique")]

selected_cols = meta_cols + annuelle_cols
df_pivot_sorted_annual = df_pivot_sorted.loc[:, selected_cols]

def format_value(val):
    try:
        if isinstance(val, str) and val.strip().startswith("%"):
            num = float(val.strip().replace("%", "").replace(",", "."))
            return "{:.2f} %".format(num)

        if isinstance(val, str) and "%" in val:
            num = float(val.replace("%", "").replace(",", ".").strip())
            return "{:.2f} %".format(num)

        if isinstance(val, (float, np.floating, int, np.integer)):
            if float(val) == int(val):
                return int(val)
            else:
                return "{:.2f}".format(float(val))

        if isinstance(val, str):
            num = float(val.replace(",", ".").strip())
            if num == int(num):
                return int(num)
            else:
                return "{:.2f}".format(num)
        return val
    except:
        return val

for col in df_pivot_sorted_annual.columns:
    df_pivot_sorted_annual[col] = df_pivot_sorted_annual[col].apply(format_value)

df_pivot_sorted_annual

C:\Users\Amsata\AppData\Local\Temp\ipykernel_26440\3059348344.py:66: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_pivot_sorted = df_pivot_sorted.sort_values('__sort_key').drop(columns='__sort_key').reset_index(drop=True)


Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire       1   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
146                   Chiffre d'affaire      10   
147                   Chiffre d'affaire      11   
148                               Marge      12   
149                              Charge      13   
150                               Marge      14   

Annee                                     Ligne_Analytique     2022     2023  \
Contexte                                                       Réel     Réel   
Mois                                                             12       12   
Nature de l'écriture                                       Annuelle Annuelle   
0                                                 RECETTES  1856015  2065453   
1                          Loyers logements et parkings HT  1715744  1908910   
2                                     CA Locatif Estudines  1709962  1903970   
3                                      CA Locatif Parkings     5782     4940   
4                                         RECETTES ANNEXES   129946   182250   
..                                                     ...      ...      ...   
146                                 % DES RECETTES TOTALES     2.11    12.11   
147                   Dotations aux amortissements (CAPEX)    14788    25150   
148                                                 EBITDA    54004   275367   
149                                                  CAPEX    95452    34342   
150                                         FREE CASH FLOW   -41447   241025   

Annee                    2024      2025              2026  
Contexte                 Réel Prévision   Budget   Budget  
Mois                       12        12       12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle Annuelle  
0                     2193409   2182306  2205868  2221683  
1                     2026364   2023357  2038521  2057395  
2                     2019099   2017043  2031296  2050195  
3                        7265      6315     7224     7200  
4                      203148    171106   179398   176470  
..                        ...       ...      ...      ...  
146                     10.22      8.33    10.08    13.40  
147                     28095     23186    27076    34447  
148                    252303    204917   249344   332103  
149                     15964     19270        0        0  
150                    236339    185646   249344   332103  

[151 rows x 9 columns]

# Test

In [28]:
data = await execute_sp(
    "ia.sp_simBudFormSA_one", 
    {
        "user_fk": config.USER_FK, 
        "sa_fk": 224, 
        "form_fk": 167
    }
)
json_string = data[0].get('EcrituresDetails')

data_records = json.loads(json_string)
context_data = pd.DataFrame(data_records)
context_data

,Section analytique,Liste de sélection,Formulaire,Lignes,Type de lignes,Compte reporting,Donnée opérationnelle,Colonnes,Type de colonnes,Montant,Cumul,Nature de l'écriture,Date,Année,Mois,Contexte
0,11000099 - Noisy,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Écart (Budget 25- Réel 24),Colonne variation,-0.77,0.0,Mensuelle,NaN,NaN,NaN,NaN
1,11000099 - Noisy,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Écart (Budget 25 - Réel 19),Colonne variation,10.08,0.0,Mensuelle,NaN,NaN,NaN,NaN
2,11000099 - Noisy,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Budget 25- Réel 24 (%),Colonne variation,1.12,0.0,Mensuelle,NaN,NaN,NaN,NaN
3,11000099 - Noisy,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Écart (Budget 25 - Réel 19),Colonne variation,70.51,0.0,Mensuelle,NaN,NaN,NaN,NaN
4,11000099 - Noisy,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Écart (Budget 25- Réel 24),Colonne variation,0.78,0.0,Mensuelle,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8003,11000099 - Noisy,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Réel 2023,Année contexte,1195.67,0.0,Mensuelle,2023-04-30,2023.0,4.0,R
8004,11000099 - Noisy,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Réel 2023,Année contexte,811.96,0.0,Mensuelle,2023-05-31,2023.0,5.0,R
8005,11000099 - Noisy,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Réel 2023,Année contexte,168.42,0.0,Mensuelle,2023-06-30,2023.0,6.0,R
8006,11000099 - Noisy,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Réel 2023,Année contexte,292.00,0.0,Mensuelle,2023-08-31,2023.0,8.0,R


In [38]:
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)
df = preprocessing_data(context_data, simple_dict, colonne_type = "Colonne variation")

In [39]:
df_CV = df[["Section  analytique", "Groupe", "Code Hiérarchique", "Lignes", "Montant", "Colonnes"]]

# Renommage et nettoyage
df = df_CV.rename(
    columns={
        'Code Hiérarchique': 'Code_H', 
        'Montant': 'Montant',
        'Lignes': 'Ligne_Analytique',
        'Groupe': 'Groupe',
        'Section  analytique': 'Residence'
    }
)

df_agg = df.groupby(
    [
        'Residence', 'Colonnes', 'Code_H', 'Ligne_Analytique'
    ]
)['Montant'].sum().reset_index()

df_pivot = df_agg.pivot_table(
    index=['Code_H', 'Ligne_Analytique'],
    columns=["Colonnes"],
    values='Montant',
    fill_value="N/A",
    aggfunc='sum'
)

df_pivot = df_pivot[[
    'Écart (Budget 25- Réel 24)', 
    'Budget 25- Réel 24 (%)'
]]

C:\Users\Amsata\AppData\Local\Temp\ipykernel_26440\3528549882.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pivot = df_agg.pivot_table(


In [ ]:
# ------------ clés de tri hiérarchique ------------
def code_hierarchical_sort_key_level(code: Any):
    s = str(code).strip().strip('.')
    if s == "" or s.lower() in ["nan", "none"]:
        return (9999,)

    m = re.fullmatch(r'(\d+)', s)
    if m:
        return (int(m.group(1)),)

    return (9999,)

# ------------ formatage pour affichage ------------
def format_numeric_for_display(series: 'pd.Series') -> 'pd.Series':
    import numpy as np
    col = series.name if hasattr(series, 'name') else ''
    try:
        vals = series.dropna().astype(float)
    except Exception:
        vals = pd.to_numeric(series, errors="coerce").dropna()
    if vals.empty:
        return series.astype(object).apply(lambda _: "N/A")
    abs_max = float(np.nanmax(np.abs(vals)))
    looks_like_pct = ('(%)' in str(col) or '%' in str(col)) or (abs_max > 1.5 and abs_max < 1000)

    def fmt(x):
        if pd.isna(x):
            return "N/A"
        try:
            x = float(x)
        except Exception:
            return str(x)
        if looks_like_pct:
            return f"{x:.2f}"
        if abs(x - round(x)) < 1e-6 and abs(x) >= 1:
            s = f"{int(round(x)):,}".replace(",", " ")
            return f"{s}"
        s = f"{x:,.2f}".replace(",", " ").replace(".", ",")
        return s

    return series.apply(fmt)

# ------------ fonction principale corrigée ------------
def sort_and_format_pivot_level(df_pivot: 'pd.DataFrame'):
    """
    Trie et filtre par Code_H hiérarchique (deux premiers niveaux), formatte les colonnes numériques.
    Ne garde que les lignes dont le Code_H a exactement deux niveaux (ex: « 5.1. », « 10.3. »)
    """
    import pandas as pd
    import numpy as np

    df = df_pivot.copy()

    # Determine le DataFrame à trier selon emplacement de Code_H
    if isinstance(df.index, pd.MultiIndex) and "Code_H" in df.index.names:
        df_reset = df.reset_index()
        code_series = df_reset["Code_H"].astype(str)
    elif "Code_H" in df.columns:
        df_reset = df.copy()
        code_series = df_reset["Code_H"].astype(str)
    else:
        # Code_H absent : juste formatage sans tri
        df_display_sorted = df.copy()
        numeric_cols = df_display_sorted.select_dtypes(include=[np.number]).columns.tolist()
        for col in numeric_cols:
            df_display_sorted[col] = format_numeric_for_display(df_display_sorted[col])
        return df_display_sorted

    # Sélectionne ceux qui ont exactement deux niveaux
    def is_level(code):
        s = str(code).strip().strip(".")
        parts = [p for p in re.split(r'[^\d]+', s) if p]
        return len(parts) == 1

    mask_level2 = code_series.apply(is_level)
    df_reset = df_reset[mask_level2].copy()
    code_series = code_series[mask_level2].copy()

    # Clé de tri hiérarchique sur 2 niveaux
    sort_keys = code_series.apply(code_hierarchical_sort_key_level)
    df_reset['__sort_key'] = sort_keys

    # Tri et remise en forme
    df_sorted = df_reset.sort_values('__sort_key').drop(columns='__sort_key').reset_index(drop=True)

    # Formatage intelligent
    numeric_cols = df_sorted.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        df_sorted[col] = format_numeric_for_display(df_sorted[col])

    # Conservation des colonnes d'origine (si pivot)
    if (set(['Groupe', 'Code_H', 'Ligne_Analytique']) <= set(df_sorted.columns)):
        index_cols = ['Groupe', 'Code_H', 'Ligne_Analytique']
        df_display_sorted = df_sorted.set_index(index_cols)
    else:
        df_display_sorted = df_sorted

    return df_display_sorted

# ------------- Exemple d'appel -------------
df_display_sorted = sort_and_format_pivot_level(df_pivot)
df_display_sorted = df_display_sorted[["Ligne_Analytique", "Écart (Budget 25- Réel 24)", "Budget 25- Réel 24 (%)"]]
print(df_display_sorted.to_markdown(index = False))

| Ligne_Analytique                     | Écart (Budget 25- Réel 24)   |   Budget 25- Réel 24 (%) |
|:-------------------------------------|:-----------------------------|-------------------------:|
| RECETTES                             | 12 459                       |                     0.57 |
| CHARGES D'IMMEUBLE DIRECTES          | -13 296                      |                    -2    |
| MARGE 1                              | 25 756                       |                     1.68 |
| Total 1                              | 40 334                       |                     3.71 |
| MARGE 2                              | -14 579                      |                    -3.3  |
| Total 2                              | -12 639                      |                    -5.83 |
| MARGE 3                              | -1 939                       |                    -0.86 |
| Dotations aux amortissements (CAPEX) | -1 019                       |                    -3.63 |
| EBITDA  

---

In [6]:
data = await execute_sp(
    "ia.sp_simBudFormSA_one", 
    {
        "user_fk": config.USER_FK, 
        "sa_fk": 8, 
        "form_fk": 167
    }
)
json_string = data[0].get('EcrituresDetails')

data_records = json.loads(json_string)
context_data = pd.DataFrame(data_records)
context_data.head()

,Section analytique,Liste de sélection,Formulaire,Lignes,Type de lignes,Compte reporting,Donnée opérationnelle,Colonnes,Type de colonnes,Cumul,Nature de l'écriture,Montant,Date,Année,Mois,Contexte
0,,AREF - Liste des résidences,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Budget 25 - Réel 19 (%),Colonne variation,0.0,Mensuelle,NaN,NaN,NaN,NaN,NaN
1,,AREF - Liste des résidences,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Écart (Budget 25 - Réel 19),Colonne variation,0.0,Mensuelle,12.67,NaN,NaN,NaN,NaN
2,,AREF - Liste des résidences,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Budget 25- Réel 24 (%),Colonne variation,0.0,Mensuelle,2.54,NaN,NaN,NaN,NaN
3,,AREF - Liste des résidences,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Budget 25- Réel 24 (%),Colonne variation,0.0,Mensuelle,-7.52,NaN,NaN,NaN,NaN
4,,AREF - Liste des résidences,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Écart (Budget 25- Réel 24),Colonne variation,0.0,Mensuelle,-1.03,NaN,NaN,NaN,NaN


In [ ]:
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)
df = preprocessing_data(context_data, simple_dict)

if df["Section  analytique"].unique().tolist() in [[''], [], None]:
    df["Section  analytique"] = df["Liste de sélection"]
df